In [9]:
import numpy as np 
import pickle 
import os 
import re 

In [ ]:
'''
Implements a greedy heuristic for hangman given a dictionary of words. 
Does not generalize to new words. 

=======================
Objects
=======================
Two dictionaries are used to store the words

word_length: {<word> : <length of word>}
length_word: {<length>: array([word1, word2, word3, ...])}


=======================
Methods
=======================

insert(String word): updates the two dictionaries with a word 

charFrequency(String[] list): calculates frequency of characters in a list

remainingCharFrequency(String[] list, char[] list): removes characters that are already successful 

remove(char c): remove all words that contain a specified character 

match(String[] list, regex): returns a list with all regex matches 

greedy_guess(String[] list): returns the most frequent character from a list of words 


'''

In [200]:
# dictionary = {<String>:<number}
word_length = {}
length_word = {}

In [201]:
def charFrequency(list_of_words): 
    dict_of_characters = {}
    for item in list_of_words: 
        arr_char = list(item)
        for x in arr_char: 
            dict_of_characters[x] = dict_of_characters.get(x, 0) + 1
    return dict_of_characters 
        

In [202]:
def remainingCharFrequency(list_of_words, successful_char): 
    dict_of_characters = charFrequency(list_of_words)
    for char in successful_char:
        del dict_of_characters[char]

In [203]:
def remove(list_of_words, char):
    list_of_matches = []
    pattern = r'(.*' + char + '.*)'
    for item in list_of_words:
        x = re.match(pattern, item)
        if x == None:
            list_of_matches.append(item)
    return list_of_matches

In [204]:
def insert(word):
    # check if the word is duplicate 
    a = word_length.get(word, 0)
    if a == 0:
        N = len(word)
        # insert in word_length
        word_length[word] = N 
        # insert in length_word 
        li1 = length_word.get(N, 0)
        if li1 != 0: 
            li1.append(word)
            length_word[N] = li1
        else:
            length_word[N] = [word]


In [253]:
def match(list_of_words, pattern):
    list_of_matches = []
    for item in list_of_words: 
        x = re.search(pattern, item)
        if x != None:
            list_of_matches.append(item)
    return list_of_matches 

In [206]:
def greedy_guess(list_of_words):
    freq_char = charFrequency(list_of_words)
    freq_char_keys = list(freq_char.keys())
    freq_char_values = list(freq_char.values())
    maxIndex = np.where(freq_char_values == np.amax(freq_char_values))[0]
    return freq_char_keys[np.random.choice(maxIndex)]

In [ ]:
'''
=================================================
HOW TO PLAY HANGMAN WITHOUT PENALTIES 
=================================================

SECRET_WORD = 'hangman'

>>>>>>>>>>  SECRET_WORD ONLY HAS LOWER CASE LETTERS <<<<<<<<<

INPUT: N = LENGTH OF SECRET_WORD

Step 1. list_of_words = length_word[N]

Step 2. chars_used = []

Step 3. x = greedy_guess(list_of_words)
        chars_used.append(x)
        if x NOT in SECRET_WORD:
            list_of_words = remove(x)
        if x in SECRET_WORD:
            create pattern 
            list_of_words = pattern(x)
Step 4. repeat Step 3
'''

In [57]:
SECRET_WORD = 'hangman'
n = len(SECRET_WORD)
# pattern = r'^([a-z]{' + re.escape(str(n)) + '})$'
pattern = r'^(.{' + re.escape(str(n)) + '})$'
re.search(pattern, 'asdasda').groups()

('asdasda',)

In [ ]:
class Dog:

    # Class Attribute
    species = 'mammal'

    # Initializer / Instance Attributes
    def __init__(self, name, age):
        self.name = name
        self.age = age


# Instantiate the Dog object
philo = Dog("Philo", 5)
mikey = Dog("Mikey", 6)

# Access the instance attributes
print("{} is {} and {} is {}.".format(
    philo.name, philo.age, mikey.name, mikey.age))

# Is Philo a mammal?
if philo.species == "mammal":
    print("{0} is a {1}!".format(philo.name, philo.species))

In [321]:
class hangman: 
    
    # Constructor 
    # Instance variables = [word, length, regex pattern, dictionary of characters, 
    #                       list of characters, number of wrong guesses]
    def __init__(self, word): 
        self.word = word # secret word 
        self.length = len(word) #length of word 
        self.pattern = r'^(.{' + re.escape(str(self.length)) + '})$' # regex pattern of game
        self.charDict = {} # dictionary of all attempted characters
        self.charList = [] # list of successful characters 
        for x in range(self.length):
            self.charList.append('.') 
        self.wrong_guesses = 0 # number of wrong guesses 
    
    # attempt a character 
    # increment wrong guess by 1 if incorrect 
    # do not let the same letter be tried twice 
    def attempt(self, char): 
        x = self.charDict.get(char, 0)
        if x == 0: 
            self.charDict[char] = True 
            indexList = self.find(char)
            if len(indexList) == 0: 
                self.wrong_guesses += 1 
                print('bad guess')
            for index in indexList:
                self.charList[index] = self.word[index]
            self.pattern = self.arrToRegex(self.charList)

        else:
            print("you've already tried this letter")
    
    
    # find the indices at which character ch appears 
    def find(self, ch):
        return [i for i, ltr in enumerate(self.word) if ltr == ch]

    # convert a list of characters to regex 
    def arrToRegex(self, list_of_characters):
        reg = r'^('
        for item in list_of_characters: 
            reg = reg + item 
        reg = reg + ')$' 
        return reg 
            
    # return the regex pattern 
    def regex(self): 
        return self.pattern
    
    # remove all words that do not have a particular character 
    def remove(self, list_of_words, char):
        list_of_matches = []
        pattern = r'(.*' + char + '.*)'
        for item in list_of_words:
            x = re.match(pattern, item)
            if x == None:
                list_of_matches.append(item)
        return list_of_matches

    # return all regex matches from a list of words 
    def match(self, list_of_words, pattern):
        list_of_matches = []
        for item in list_of_words: 
            x = re.search(pattern, item)
            if x != None:
                list_of_matches.append(item)
        return list_of_matches 

    # frequency of characters given a list of words 
    def charFrequency(self, list_of_words): 
        dict_of_characters = {}
        for item in list_of_words: 
            arr_char = list(item)
            for x in arr_char: 
                dict_of_characters[x] = dict_of_characters.get(x, 0) + 1
        return dict_of_characters 

    # guess the next words 
    def greedy_guess(self, list_of_words):
        freq_char = self.charFrequency(list_of_words)
        temp_freq_char = self.charFrequency(list_of_words)
        for item in temp_freq_char:
            x = self.charDict.get(item, 0)
            if x != 0: 
                del freq_char[item]
        freq_char_keys = list(freq_char.keys())
        freq_char_values = list(freq_char.values())
        maxIndex = np.where(freq_char_values == np.amax(freq_char_values))[0]
        return freq_char_keys[np.random.choice(maxIndex)]
        

In [227]:
h1 = hangman('michael gintz')

In [236]:
h1.attempt('z')

In [237]:
h1.wrong_guesses

3